In [4]:
import pandas as pd
from IPython.display import display
import _1_Preprocessing

df_train, df_test = _1_Preprocessing.run_preprocessing()


size before remove outliers:  (15585053, 15)
size after remove outliers:  (14329788, 15)
size after add article total weight:  (14329788, 17)


/Users/canis/Library/CloudStorage/OneDrive-Persönlich/programme/coding/github/flaschenpost/_1_Preprocessing.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  article_id_dummies = df.groupby('web_order_id')['article_id'].apply(lambda x: pd.Series(


size after one hot encoding:  (14329788, 49)
size after handle missing values:  (14329755, 49)
size after service time start ordinal encoding:  (14329755, 49)
size after train test split:  (11463804, 49) (2865951, 49)


In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pymc as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error

# =========================
# Datenvorbereitung
# =========================

# Definiere die Featureliste (hier z.B. Artikelgewicht, Booleans, Stock, etc.)
features = [
    "article_weight_in_g", "is_business", "is_pre_order", "has_elevator", "floor", 
    "num_previous_orders_customer", "customer_speed"
]
# Ergänze Features, die mit 'crate_count_' und 'article_id_' beginnen
features += [col for col in df_train.columns if col.startswith('crate_count_')]
features += [col for col in df_train.columns if col.startswith('article_id_')]

# Extrahiere die Prädiktoren und das Target aus dem Trainingsdatensatz
X_train = df_train[features].values
y_train = df_train['service_time_in_minutes'].values

# Ebenso für den Testdatensatz
X_test = df_test[features].values
y_test = df_test['service_time_in_minutes'].values

# Skaliere die kontinuierlichen Features (wichtig für Konvergenz)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# =========================
# Gruppierungsvariablen definieren
# =========================

# Konvertiere customer_id und driver_id in numerische Codes
customer_ids_train = df_train['customer_id'].astype('category').cat.codes.values
driver_ids_train   = df_train['driver_id'].astype('category').cat.codes.values

# Für den Testdatensatz (achte darauf, dass die Kategorien konsistent sind)
customer_ids_test = df_test['customer_id'].astype('category').cat.codes.values
driver_ids_test   = df_test['driver_id'].astype('category').cat.codes.values

# Anzahl der Gruppen und Features
n_customers = len(np.unique(customer_ids_train))
n_drivers   = len(np.unique(driver_ids_train))
n_features  = X_train_scaled.shape[1]

# =========================
# Hierarchisches Bayesian Modell definieren und anpassen
# =========================

with pm.Model() as hier_model:
    # --- Globale Effekte ---
    beta0 = pm.Normal('beta0', mu=0, sigma=10)                   # Globaler Intercept
    beta  = pm.Normal('beta', mu=0, sigma=10, shape=n_features)     # Koeffizienten für die erklärenden Variablen
    
    # --- Gruppen-spezifische Effekte ---
    # Wir modellieren einen individuellen Effekt pro Customer und pro Driver.
    sigma_customer = pm.HalfNormal('sigma_customer', sigma=5)
    sigma_driver   = pm.HalfNormal('sigma_driver', sigma=5)
    
    customer_effect = pm.Normal('customer_effect', mu=0, sigma=sigma_customer, shape=n_customers)
    driver_effect   = pm.Normal('driver_effect', mu=0, sigma=sigma_driver, shape=n_drivers)
    
    # --- Erwartungswert der Beobachtungen ---
    # Lineare Kombination aus globalen Effekten, den Prädiktoren sowie den gruppenspezifischen Effekten
    mu = beta0 + pm.math.dot(X_train_scaled, beta) + customer_effect[customer_ids_train] + driver_effect[driver_ids_train]
    
    # --- Likelihood ---
    sigma = pm.HalfNormal('sigma', sigma=5)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_train)
    
    # --- Sampling aus dem Posterior ---
    trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

# =========================
# Vorhersagen auf dem Testdatensatz berechnen
# =========================

# Wir extrahieren aus dem Trace die Posterior-Samples
beta0_samples = trace.posterior['beta0'].values.reshape(-1)  # Globaler Intercept
beta_samples = trace.posterior['beta'].values.reshape(-1, n_features)  # Koeffizienten
customer_effect_samples = trace.posterior['customer_effect'].values.reshape(-1, n_customers)
driver_effect_samples = trace.posterior['driver_effect'].values.reshape(-1, n_drivers)

# Gesamtzahl der Posterior-Samples
n_samples = beta0_samples.shape[0]
n_test = X_test_scaled.shape[0]

# Für jeden Posterior-Sample berechnen wir eine Vorhersage für alle Testbeobachtungen
predictions = np.zeros((n_samples, n_test))
for i in range(n_samples):
    predictions[i, :] = (beta0_samples[i] +
                         np.dot(X_test_scaled, beta_samples[i]) +
                         customer_effect_samples[i][customer_ids_test] +
                         driver_effect_samples[i][driver_ids_test])
    
# Für jede Testbeobachtung mitteln wir über alle Samples, um einen Punkt-Schätzwert zu erhalten
y_pred_mean = predictions.mean(axis=0)

# =========================
# Berechnung der Fehlermaße: MSE und MAE
# =========================

mse = mean_squared_error(y_test, y_pred_mean)
mae = mean_absolute_error(y_test, y_pred_mean)

print("Test Mean Squared Error (MSE):", mse)
print("Test Mean Absolute Error (MAE):", mae)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta, sigma_customer, sigma_driver, customer_effect, driver_effect, sigma]


/Users/canis/Library/CloudStorage/OneDrive-Persönlich/programme/coding/github/flaschenpost/.venv/lib/python3.11/sit
e-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

ValueError: Not enough samples to build a trace.